In [1]:
import requests
import pprint as pp
import pandas as pd
import openpyxl

pd.options.display.max_rows = 50

# user input
ejerlav = [11658, 11659]

lat_max = 55.618861
lat_min = 55.6024694
long_max = 12.582397
long_min = 12.573047

areal_min = 725
areal_max = 2000

In [2]:
def get_endpoint_data(endpoint):
    response = requests.get(endpoint)
    status = response.status_code

    if status == 200:
        result = response.json()
        if len(result) == 0:
            result =  None
    else:
        result = None
    
    return result

def api_jordstykke_matrikel(ejerlav, matrikelnr=None):
    
    if matrikelnr is None:
        endpoint = f'https://api.dataforsyningen.dk/jordstykker?ejerlavkode={ejerlav}'
    else:
        endpoint = f'https://api.dataforsyningen.dk/jordstykker?ejerlavkode={ejerlav}&matrikelnr={matrikelnr}'
    
    return get_endpoint_data(endpoint)
    

def api_adresse_matrikel(ejerlav, matrikelnr=None):
    if matrikelnr is None:
        endpoint = f'https://api.dataforsyningen.dk/adresser?ejerlavkode={ejerlav}'#'&struktur=mini'
    else:
        endpoint = f'https://api.dataforsyningen.dk/adresser?ejerlavkode={ejerlav}&matrikelnr={matrikelnr}'
    
    return get_endpoint_data(endpoint)


def api_adresse_adresse(vej, husnr):
    endpoint = f'https://api.dataforsyningen.dk/adresser?vejnavn={vej}&husnr={husnr}'
        
    return get_endpoint_data(endpoint)


def api_bbr(adresseid):
    endpoint = f'https://api.dataforsyningen.dk/bbrlight/enheder?adresseid={adresseid}'
    
    return get_endpoint_data(endpoint)


def opslag_enkelt_bolig(vej, husnr):
    adresse = api_adresse_adresse(vej, husnr)
    id_adresse = adresse[0]['id']
    bbr = api_bbr(id_adresse)
    return bbr


In [3]:
# Jordstykke data
def jordstykke_data(ejerlav):

    data_jord_raw = pd.DataFrame()
    
    for el in ejerlav:
        results = api_jordstykke_matrikel(ejerlav=el, matrikelnr=None)
        data_jord_raw = data_jord_raw.append(results, ignore_index=True)
    
    cols_keep = ['matrikelnr', 'visueltcenter', 'ejerlav', 'registreretareal']
    data_jord = data_jord_raw[cols_keep].copy()

    data_jord['ejerlav_kode'] = data_jord['ejerlav'].apply(lambda x: x['kode'])
    data_jord['long'] = data_jord['visueltcenter'].apply(lambda x: x[0])
    data_jord['lat'] = data_jord['visueltcenter'].apply(lambda x: x[1])

    data_jord = data_jord.drop(columns=['ejerlav', 'visueltcenter'])
    data_jord = data_jord.rename(columns={'ejerlav_kode': 'ejerlav',
                                            'matrikelnr': 'matrikel',
                                            'registreretareal': 'areal'})
    return data_jord

data_jord = jordstykke_data(ejerlav=ejerlav)
data_jord

,matrikel,areal,ejerlav,long,lat
0,100,444,11658,12.598014,55.627497
1,101,1315,11658,12.584378,55.620139
2,102,1398,11658,12.593118,55.627143
3,103a,461,11658,12.595771,55.632187
4,103b,262,11658,12.597426,55.632120
...,...,...,...,...,...
5543,9u,1058,11659,12.582011,55.601482
5544,9v,665,11659,12.583093,55.600063
5545,9x,640,11659,12.583279,55.600217
5546,9y,640,11659,12.583304,55.600360


In [4]:
# Adresse data
def adresse_data(ejerlav):

    data_adr_raw = pd.DataFrame()
    
    for el in ejerlav:
        data_adr_raw = data_adr_raw.append(api_adresse_matrikel(ejerlav=el,matrikelnr=None))
        
    cols_keep = ['id', 'adgangsadresse']
    data_adr = data_adr_raw[cols_keep].copy()

    data_adr['ejerlav_kode'] = data_adr['adgangsadresse'].apply(lambda x: x['ejerlav']['kode'])
    data_adr['matrikelnr'] = data_adr['adgangsadresse'].apply(lambda x: x['matrikelnr'])
    data_adr['vej'] = data_adr['adgangsadresse'].apply(lambda x: x['vejstykke']['navn'])
    data_adr['husnr'] = data_adr['adgangsadresse'].apply(lambda x: x['husnr'])

    data_adr = data_adr.drop(columns=['adgangsadresse'])
    data_adr = data_adr.rename(columns={'id': 'id_adresse', 
                                        'ejerlav_kode': 'ejerlav',
                                        'matrikelnr': 'matrikel'})

    return data_adr
    
data_adr = adresse_data(ejerlav=ejerlav)
data_adr

,id_adresse,ejerlav,matrikel,vej,husnr
0,6986b5b3-2cd6-46e5-825e-88ce4b132cb2,11658,100,Smedekærvej,19C
1,730730ec-dd6c-46e1-b515-b01437189dc6,11658,106,Brændekildevej,1A
2,0a3f50a7-6d46-32b8-e044-0003ba298018,11658,10aa,Talosvej,7
3,0a3f50a7-6d44-32b8-e044-0003ba298018,11658,10ab,Talosvej,5
4,0a3f50a7-74b8-32b8-e044-0003ba298018,11658,10ad,Vestavej,2
...,...,...,...,...,...
3160,123d736c-6de9-4658-a9df-cfe15cadd8d1,11659,9t,Kamerunvej,17
3161,0a3f50a7-50e2-32b8-e044-0003ba298018,11659,9v,Kongelundsvej,469
3162,0a3f50a7-50e0-32b8-e044-0003ba298018,11659,9x,Kongelundsvej,467
3163,0a3f50a7-50de-32b8-e044-0003ba298018,11659,9y,Kongelundsvej,465


In [5]:
# Merge adresse og jorddata
data = data_adr.merge(data_jord,
                      how='inner',
                      on=['ejerlav', 'matrikel'])
data

,id_adresse,ejerlav,matrikel,vej,husnr,areal,long,lat
0,6986b5b3-2cd6-46e5-825e-88ce4b132cb2,11658,100,Smedekærvej,19C,444,12.598014,55.627497
1,730730ec-dd6c-46e1-b515-b01437189dc6,11658,106,Brændekildevej,1A,138,12.593194,55.628465
2,0a3f50a7-6d46-32b8-e044-0003ba298018,11658,10aa,Talosvej,7,785,12.588948,55.620657
3,0a3f50a7-6d44-32b8-e044-0003ba298018,11658,10ab,Talosvej,5,739,12.588788,55.620862
4,0a3f50a7-74b8-32b8-e044-0003ba298018,11658,10ad,Vestavej,2,905,12.587129,55.620849
...,...,...,...,...,...,...,...,...
8547,123d736c-6de9-4658-a9df-cfe15cadd8d1,11659,9t,Kamerunvej,17,6722,12.581378,55.601219
8548,0a3f50a7-50e2-32b8-e044-0003ba298018,11659,9v,Kongelundsvej,469,665,12.583093,55.600063
8549,0a3f50a7-50e0-32b8-e044-0003ba298018,11659,9x,Kongelundsvej,467,640,12.583279,55.600217
8550,0a3f50a7-50de-32b8-e044-0003ba298018,11659,9y,Kongelundsvej,465,640,12.583304,55.600360


In [6]:
# Afgræns boligområde

mask_lat = data['lat'].between(left=lat_min, right=lat_max)
mask_long = data['long'].between(left=long_min, right=long_max)

mask_areal = (data['areal'] >= areal_min) & (data['areal'] <= areal_max)

df_export = data[mask_lat & mask_long & mask_areal].reset_index(drop=True)

df_export

,id_adresse,ejerlav,matrikel,vej,husnr,areal,long,lat
0,0a3f50a7-6b60-32b8-e044-0003ba298018,11658,11dæ,Studsbøl Alle,33,784,12.581112,55.617727
1,0a3f50a7-39a8-32b8-e044-0003ba298018,11658,11du,Finderupvej,85,794,12.580802,55.618410
2,0a3f50a7-5a5e-32b8-e044-0003ba298018,11658,11dv,Nybøl Alle,28,774,12.582138,55.616157
3,0a3f50a7-5a5f-32b8-e044-0003ba298018,11658,11dx,Nybøl Alle,30,752,12.582130,55.615990
4,0a3f50a7-6b68-32b8-e044-0003ba298018,11658,11dy,Studsbøl Alle,41,1098,12.581371,55.617087
...,...,...,...,...,...,...,...,...
260,0a3f50a7-6058-32b8-e044-0003ba298018,11659,7ah,Rhodesiavej,17,744,12.578047,55.605364
261,0a3f50a7-6056-32b8-e044-0003ba298018,11659,7ai,Rhodesiavej,15,735,12.577796,55.605510
262,0a3f50a7-6054-32b8-e044-0003ba298018,11659,7ak,Rhodesiavej,13,726,12.578063,55.605642
263,0a3f50a7-5a43-32b8-e044-0003ba298018,11659,7ar,Nubiensvej,38,872,12.578270,55.606868


In [ ]:
# Fecth BBR info
def append_bbr_info(df):
    df['bbr'] = df['id_adresse'].apply(lambda x: api_bbr(x))
    
    mask_bbr_null = df['bbr'].notnull()

    df.loc[mask_bbr_null, 'BEBO_ARL'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['BEBO_ARL'])
    df.loc[mask_bbr_null, 'ENH_ARL_SAML'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['ENH_ARL_SAML'])
    df.loc[mask_bbr_null, 'VAERELSE_ANT'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['VAERELSE_ANT'])
    df.loc[mask_bbr_null, 'AntBadevaerelser'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['AntBadevaerelser'])
    df.loc[mask_bbr_null, 'OPFOERELSE_AAR'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['bygning']['OPFOERELSE_AAR'])
    df.loc[mask_bbr_null, 'OMBYG_AAR'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['bygning']['OMBYG_AAR'])
    df.loc[mask_bbr_null, 'ETAGER_ANT'] = df.loc[mask_bbr_null, 'bbr'].apply(lambda x: x[0]['bygning']['ETAGER_ANT'])
  
    return df

df_export = append_bbr_info(df_export)
df_export

In [ ]:
cols_ignore = ['id_adresse','ejerlav','matrikel', 'long', 'lat', 'area_type', 'bbr']

df_export.loc[:,~df_export.columns.isin(cols_ignore)].to_excel('store_grunde.xlsx', index=False)